This notebook assumes that a connection has been made with the 

In [ ]:
import IPython.display

import os.path

These are the tables that we need to explicitly drop:

In [ ]:
tables_to_drop_ls=['patient', 'drug', 'doctor', 
                   'prescription', 'female_patient',
                   'young_patient', 'patient_no_pk',
                   'test_table', 'customer', 'disk_order',
                   'distributor_item', 'drug_order', 
                   'order_item', 'order_line', 
                   'unnormalised_prescription', 'distributor',
                   'item']

See which tables have been defined by the current user:

In [ ]:
%%capture --no-display capture_output
%%sql users_tables <<
    SELECT tablename
    FROM pg_tables
    WHERE tableowner=(SELECT current_user)
        AND schemaname='public'

In [ ]:
if not users_tables.empty:
    users_tables=list(users_tables['tablename'])
else:
    users_tables=[]

Go through `tables_to_drop_ls`, and drop all the tables in that list, using `CASCADE` to remove any further dependencies:

In [ ]:
%%capture --no-display capture_output

for table in tables_to_drop_ls:

    if table in users_tables:
        IPython.display.display_pretty('Removing table {}'.format(table))
        %sql DROP TABLE IF EXISTS $table CASCADE



Now we need to set up the hospital schema. Drop the schema if it already exists:

In [ ]:
IPython.display.display_pretty('Reset hospital schema')

In [ ]:
%%capture capture_output
%%sql

DROP SCHEMA IF EXISTS hospital CASCADE;

Now recreate the schema:

In [ ]:
%%capture capture_output
%%sql

CREATE SCHEMA hospital;

...and create the tables needed for the hospital database:

In [ ]:
%%capture capture_output
%%sql

CREATE TABLE hospital.patient (
    patient_id CHAR(4),
    patient_name VARCHAR(20),
    date_of_birth DATE,
    gender CHAR(6),
    height_cm DECIMAL(4,1),
    weight_kg DECIMAL(4,1),
    doctor_id CHAR(4));

In [ ]:
%%capture capture_output
%%sql

CREATE TABLE hospital.doctor (
    doctor_id CHAR(4),
    doctor_name VARCHAR(20));

In [ ]:
%%capture capture_output
%%sql

 CREATE TABLE hospital.drug (
     drug_code CHAR(6),
     drug_name TEXT)

In [ ]:
%%capture capture_output
%%sql

CREATE TABLE hospital.prescription (
    patient_id CHAR(4),
    doctor_id VARCHAR(20),
    drug_code CHAR(6),
    date DATE,
    dosage TEXT,
    duration TEXT);

Add the primary keys:

In [ ]:
%%capture capture_output
%%sql

ALTER TABLE hospital.patient
    ADD CONSTRAINT hospital_patient_pk
        PRIMARY KEY (patient_id);

In [ ]:
%%capture capture_output
%%sql

ALTER TABLE hospital.doctor
    ADD CONSTRAINT hospital_doctor_pk
        PRIMARY KEY (doctor_id);

In [ ]:
%%capture capture_output
%%sql

ALTER TABLE hospital.drug
    ADD CONSTRAINT hospital_drug_pk
        PRIMARY KEY (drug_code);

In [ ]:
%%capture capture_output
%%sql

ALTER TABLE hospital.prescription
    ADD CONSTRAINT hospital_prescription_pk
        PRIMARY KEY (patient_id, doctor_id, drug_code, date);

Populate the tables:

In [ ]:
%%capture capture_output

# hospital.doctor

df=pd.read_csv(os.path.join('.', 'sql_data', 'doctor.csv'))

for (id, name) in df.itertuples(index=False, name=None):
    %sql INSERT INTO hospital.doctor (doctor_id, doctor_name) VALUES (:id, :name);


In [ ]:
%%capture capture_output

# hospital.patient

df=pd.read_csv(os.path.join('.', 'sql_data', 'patient.csv'),
               parse_dates=['date_of_birth'])

for (v1, v2, v3, v4, v5, v6, v7) in df.itertuples(index=False, name=None):
    
    %sql INSERT INTO hospital.patient (patient_id, patient_name, date_of_birth, gender, height_cm, weight_kg, doctor_id) VALUES (:v1, :v2, :v3, :v4, :v5, :v6, :v7);



In [ ]:
%%capture capture_output

# hospital.drug

df=pd.read_csv(os.path.join('.', 'sql_data', 'drug.csv'))

for (code, name) in df.itertuples(index=False, name=None):
    %sql INSERT INTO hospital.drug (drug_code, drug_name) VALUES (:code, :name);

In [ ]:
%%capture capture_output

# hospital.prescription

df=pd.read_csv(os.path.join('.', 'sql_data', 'prescription.csv'),
               parse_dates=['date'])

for (v1, v2, v3, v4, v5, v6) in df.itertuples(index=False, name=None):
    %sql INSERT INTO hospital.prescription (patient_id, date, doctor_id, drug_code, dosage, duration) VALUES (:v1, :v2, :v3, :v4, :v5, :v6);


Finally, add the foreign key constraints

In [ ]:
%%capture capture_output
%%sql

ALTER TABLE hospital.patient
    ADD CONSTRAINT hospital_patient_doctor_fk
        FOREIGN KEY (doctor_id) REFERENCES hospital.doctor;

In [ ]:
%%capture capture_output
%%sql

ALTER TABLE hospital.prescription
    ADD CONSTRAINT hospital_prescription_patient_fk
        FOREIGN KEY (patient_id) REFERENCES hospital.patient;

In [ ]:
%%capture capture_output
%%sql

ALTER TABLE hospital.prescription
    ADD CONSTRAINT hospital_prescription_doctor_fk
        FOREIGN KEY (doctor_id) REFERENCES hospital.doctor;

In [ ]:
%%capture capture_output
%%sql

ALTER TABLE hospital.prescription
    ADD CONSTRAINT hospital_prescription_drug_fk
        FOREIGN KEY (drug_code) REFERENCES hospital.drug;

And we're finished, so print a closing message:

In [ ]:
IPython.display.display_pretty('done')